<h1>Jobs analysis project</h1>
This is a project where I built a data pipeline, continuously collecting new data on data science, engineering, and analytics jobs from monster.com, then perform analyses and visualizations on how various aspects of the collected data affect salaries in the field. The focus of the project is on the pipeline itself, with the other aspects primarily being there to give the pipeline something to do. I used Julia for the entire pipeline. I used the Gumbo and Cascadia Julia libraries for web scraping, and collected and organized the data into a SQLite database.

In [1]:
using DataFrames
using SQLite
using PyCall
using ScikitLearn
@sk_import linear_model: (Ridge, Lasso)
const DATA_DIR = ".\\data\\cleaned"

".\\data\\cleaned"

In [2]:
db = SQLite.DB("$(DATA_DIR)\\jobs.db")

SQLite.DB(".\data\cleaned\jobs.db")

In [3]:
#analyze/create visualizations for geo data (put in function to avoid global variables)
function geo_analysis()
    location_data = SQLite.DBInterface.execute(db, "SELECT * FROM Geography")
    location_df = DataFrame(location_data)
end
geo_analysis()

,PostingID,Location,Salary,Date
,Int64,String,Int64?,String
1,1,"Clearfield, UT",101000,2023-01-23
2,2,"Nashville, TN",65500,2023-01-23
3,3,"Chantilly, VA",170000,2023-01-23
4,4,"Apple Valley, MN",107000,2023-01-23
5,5,"Columbus, OH",80000,2023-01-23
6,6,"Boston, MA",62500,2023-01-23
7,7,"Dallas, TX",78000,2023-01-23
8,8,"Huntsville, AL",150000,2023-01-23
9,9,"Palmdale, CA",238500,2023-01-23


In [4]:
function company_analysis()
    company_data = SQLite.DBInterface.execute(db, "SELECT * FROM Companies")
    company_df = DataFrame()
end
company_analysis()

""
""


In [25]:
function find_slopes(model, dims)
    zero_list = zeros(dims)
    slopes = []
    basic = predict(model, [zero_list,])[1]
    for i in range(1, dims)
        zero_list[i] = 1
        predicted = predict(model, [zero_list,])[1]
        push!(slopes, predicted - basic)
    end
    slopes
end

function title_analysis()
    special_columns = ["PostingID", "Salary", "Date"]
    data = SQLite.DBInterface.execute(db, "SELECT * FROM Title")
    title_df = DataFrame(data)
    dropmissing!(title_df)
    predictors = [column for column in names(title_df) if !in(column, special_columns)]
    predictor_df = DataFrame()
    for predictor in predictors
        predictor_df[!, predictor] = title_df[!, predictor]
    end
    ridge = Ridge()
    lasso = Lasso()
    
    #not splitting into train and test because this is just for inference
    
    ScikitLearn.fit!(ridge, Array(predictor_df), Array(title_df[!, "Salary"]))
    ScikitLearn.fit!(lasso, Array(predictor_df), title_df[!, "Salary"])
    
    ridge_slopes = find_slopes(ridge, length(names(predictor_df)))
    lasso_slopes = find_slopes(lasso, length(names(predictor_df)))
    
    output_df = DataFrame(names = names(predictor_df), ridges = ridge_slopes, lassos = lasso_slopes)
    output_df[!, "Mean"] = (output_df[!, "ridges"] + output_df[!, "lassos"]) / 2
    sort!(output_df, [order(:Mean, rev = true)])
    
    output_df
end
title_analysis()

,names,ridges,lassos,Mean
,String,Any,Any,Float64
1,systems,1.60122e5,2.0948e5,1.84801e5
2,sr,1.39232e5,1.79794e5,159513.0
3,software,96786.7,1.18959e5,1.07873e5
4,senior,73992.8,90429.1,82211.0
5,principal,57810.1,83928.0,70869.1
6,scientist,63805.0,76180.5,69992.7
7,manager,46593.5,73966.4,60280.0
8,project,48020.9,67340.1,57680.5
9,science,48236.3,65137.5,56686.9


The above analyses are for the titles of the jobs postings. Selected words that were present in a particular proportion of job posting titles were used to fit a model to predict salary/wage based on words in the job title, with titles being represented as one-hot encodings. Negative slopes represent a word the presence of which indicates a lower salary relative to other salaries in the sample, and positive slopes represent an increase. Higher absolute values indicate a larger predicted effect.